In [6]:
!ls

'Code Zoo saved just in case.ipynb'
 Data_analysis_and_benchmarking.ipynb
 dog_app.ipynb
'mobilenet no preprocess no imagenet.png'
'mobilenet no preprocess.png'
'MURA ALL.ipynb'
 MURA_All_limps.ipynb
'Mura Bone Abnormality classifier.ipynb'
'MURA Clean Code.ipynb'
 MURA.pdf
'MURA report.ipynb'
 MURA-v1.1
 MURA-v1.1.zip
 Results.ipynb


In [7]:
# -*- coding: utf-8 -*-
#!wget -c https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
#!unzip MURA-v1.1.zip
#!rm MURA-v1.1.zip
!ls

'Code Zoo saved just in case.ipynb'
 Data_analysis_and_benchmarking.ipynb
 dog_app.ipynb
'mobilenet no preprocess no imagenet.png'
'mobilenet no preprocess.png'
'MURA ALL.ipynb'
 MURA_All_limps.ipynb
'Mura Bone Abnormality classifier.ipynb'
'MURA Clean Code.ipynb'
 MURA.pdf
'MURA report.ipynb'
 MURA-v1.1
 MURA-v1.1.zip
 Results.ipynb


In [8]:
### If the drive thing didn't work unzip the downloaded files to a folder and give its name to directory
# from google.colab import drive
# drive.mount('/content/drive')

** Imports**

In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import keras
pd.options.display.max_colwidth = 100
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.densenet import DenseNet169
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.applications.mobilenetv2 import preprocess_input
from keras.applications import MobileNet
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing import image as k_im_prep
from keras.models import Model
from keras import backend as K
from keras.models import Sequential

In [10]:
csv='MURA-v1.1/valid_labeled_studies.csv'
studies=pd.read_csv(csv, sep=',',header=None)

## TODO make a dataframe of images and corresponding labels

In [11]:
studies.head(2)

,0,1
0,MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/,1
1,MURA-v1.1/valid/XR_WRIST/patient11186/study1_positive/,1


In [29]:
studies=studies.rename({0:'filename',1:'labels'},axis="columns")

In [30]:
list(studies.columns.values)

['filename', 'labels']

In [12]:
def preprocess(img):
    img = k_im_prep.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    img=np.array(img)[0]
    return img

In [13]:
#should I use feature wise or sample wise center ?
train_datagen = ImageDataGenerator(  rescale=1./255,
    featurewise_center=True,  #CHANGED IT TO TRUE # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  #CHANGED IT TO TRUE# divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    zoom_range=0.1,
    channel_shift_range=0.,preprocessing_function=preprocess,
    fill_mode='nearest')

train_datagen.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
train_datagen.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]

In [14]:
#should I use feature wise or sample wise center ?
valid_datagen = ImageDataGenerator(  rescale=1./255,
    featurewise_center=True,  #CHANGED IT TO TRUE # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  #CHANGED IT TO TRUE# divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    zoom_range=0.1,
    channel_shift_range=0.,preprocessing_function=preprocess,
    fill_mode='nearest')

valid_datagen.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
valid_datagen.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]

In [31]:
train_generator=train_datagen.flow_from_dataframe(studies, directory="drive/My Drive/", x_col='filename', y_col='labels', has_ext=True,
                                                  target_size=(224, 224),color_mode='rgb', classes=None,class_mode='binary',
                                                  batch_size=32, shuffle=True, seed=None,save_to_dir=None, save_prefix='',
                                                  save_format='png', subset=None, interpolation='nearest')

ValueError: has_ext is set to True but extension not found in x_col

In [ ]:
valid_generator=valid_datagen.flow_from_dataframe(studies, directory="drive/My Drive/", x_col='filename', y_col='labels', has_ext=True,
                                                  target_size=(224, 224),color_mode='rgb', classes=None,class_mode='binary',
                                                  batch_size=32, shuffle=True, seed=None,save_to_dir=None, save_prefix='',
                                                  save_format='png', subset=None, interpolation='nearest')

In [ ]:
#test the generators

In [ ]:
GEN_BATCH=64


# which Transfer Learning base models to use
base=[2]
#this is just to know index of each model when choosing base above and to print it when running the function
bases=["MobileNetV2", "DenseNet169", "InceptionV3", "ResNet50","NASNetMobile"]


# whether to use imagenet weights of not
imagenet=True


# whether to freeze ALL layers of base model or not
freeze_all=False


# how much information to display about epochs and progress , 0= none , 1 is line per epoch
verbose=1


# whether to preprocess input 
preprocess_ip=True


#whether to augment data
augmentation=True

epoch_=5
batch=64